In [49]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
from IPython.display import display, HTML
import seaborn as sns
import random
import matplotlib.pyplot as plt
import plotly.express as px
import pandas_profiling as pp
from scipy import stats
from scipy.stats import norm
%matplotlib inline
import h2o
from h2o.automl import H2OAutoML

In [52]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('data2/landmarks.csv')
df.columns = [f'col{i}' for i in range(1405)]
df.astype({'col1404':'string'})

Y = df.iloc[:]['col1404']
X = df.drop(columns=['col1404'])
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.15)

In [62]:
ytrain.unique()

array(['surprise', 'anger', 'contempt', 'sad', 'happy', 'disgust',
       'neutral', 'fear'], dtype=object)

In [63]:
h2o.init(
    nthreads=8,     # number of threads when launching a new H2O server
    max_mem_size=8  # in gigabytes
)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,4 hours 29 mins
H2O_cluster_timezone:,Europe/Moscow
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_vorkov_s0lk7n
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.731 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [47]:
# train.groupby("col1404")["col1404"].count()

col1404
anger       3159
contempt    2857
disgust        1
happy       5038
neutral     5117
surprise    4019
Name: col1404, dtype: int64

In [65]:
train = xtrain
train["col1404"] = ytrain

test = xtest
test["col1404"] = ytest

train_h2o_frame = h2o.H2OFrame(train)
test_h2o_frame = h2o.H2OFrame(test)

x = train_h2o_frame.columns
y = "col1404"
x.remove(y)

aml = H2OAutoML(max_models=8, seed=1, max_runtime_secs=60*60, verbosity='info')
aml.train(x=x, y=y, training_frame=train_h2o_frame)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
04:36:41.602: Project: AutoML_2_20230909_43641
04:36:41.602: 5-fold cross-validation will be used.
04:36:41.603: Setting stopping tolerance adaptively based on the training frame: 0.006386420048890296
04:36:41.603: Build control seed: 1
04:36:41.604: training frame: Frame key: AutoML_2_20230909_43641_training_Key_Frame__upload_ae3d0a5414d9ea46ac6abb67ac324507.hex    cols: 1405    rows: 24518  chunks: 157    size: 290473276  checksum: 9014670398134974866
04:36:41.604: validation frame: NULL
04:36:41.604: leaderboard frame: NULL
04:36:41.604: blending frame: NULL
04:36:41.604: response column: col1404
04:36:41.604: fold column: null
04:36:41.604: weights column: null
04:36:41.608: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 90

Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_1_AutoML_2_20230909_43641


Model Summary: 
    number_of_trees
--  -----------------
    40

ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.06863666117708979
RMSE: 0.261985994238413
LogLoss: 0.2527321984390449
Mean Per-Class Error: 0.01463669311478975
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
anger    contempt    disgust    fear    happy    neutral    sad    surprise    Error       Rate
-------  ----------  ---------  ------  -------  ---------  -----  ----------  ----------  ------------
2666     2           1          7       0        2          9      10          0.0114943   31 / 2,697
7        2381        5          4       5        22         0      3           0.0189534   46 / 2,427
4        5           2072       5       12       13         5      9           0.0249412   53 / 2,125
7        6           5          2597    2        1          7      5           0.0125475   33 / 2,630
1        7           7          0       4254     19         3      7           0.0102373   44 / 4,298
17       8           4          9       6        4308       13     8           0.0148639   65 / 4,373
3        4           2          7       1        1          2550   6           0.00932401  24 / 2,574
7        5           1          14      2        18         3      3344        0.0147319   50 / 3,394
2712     2418        2097       2643    4282     4384       2590   3392        0.0141121   346 / 24,518

Top-8 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.985888
2    0.996615
3    0.998369
4    0.999062
5    0.999225
6    0.999674
7    0.999959
8    1

ModelMetricsMultinomial: xgboost
** Reported on cross-validation data. **

MSE: 0.38721573446474844
RMSE: 0.6222666104369963
LogLoss: 1.190552812564536
Mean Per-Class Error: 0.5015266423843869
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
anger    contempt    disgust    fear    happy    neutral    sad    surprise    Error     Rate
-------  ----------  ---------  ------  -------  ---------  -----  ----------  --------  ---------------
1094     332         329        263     6        5          431    237         0.594364  1,603 / 2,697
306      1258        161        94      19       64         276    249         0.481665  1,169 / 2,427
520      276         495        242     21       24         294    253         0.767059  1,630 / 2,125
301      131         180        921     2        4          317    774         0.64981   1,709 / 2,630
3        13          13         10      3728     495        7      29          0.13262   570 / 4,298
19       23          15         11      351      3806       36     112         0.129659  567 / 4,373
460      337         227        333     2        7          771    437         0.700466  1,803 / 2,574
231      214         151        677     50       222        344    1505        0.55657   1,889 / 3,394
2934     2584        1571       2551    4179     4627       2476   3596        0.446203  10,940 / 24,518

Top-8 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.553797
2    0.740884
3    0.842932
4    0.914104
5    0.961498
6    0.98964
7    0.998124
8    1

Cross-Validation Metrics Summary: 
                         mean      sd      

In [24]:
aml.distribution

'AUTO'

In [66]:
lb = aml.leaderboard
lb.head(rows=15)

model_id,mean_per_class_error,logloss,rmse,mse
XGBoost_1_AutoML_2_20230909_43641,0.501527,1.19055,0.622267,0.387216


In [67]:
test_h2o_frame = test_h2o_frame.drop('col1404')
pred = aml.predict(test_h2o_frame)

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


In [68]:
pred


predict,anger,contempt,disgust,fear,happy,neutral,sad,surprise
surprise,0.0806781,0.00519591,0.0283022,0.252751,0.000609164,0.00137734,0.0142793,0.616807
fear,0.0371579,0.00272076,0.0964443,0.67374,0.000719777,0.00147301,0.00763326,0.180111
neutral,0.00535385,0.000793096,0.00281587,0.0123343,0.00806528,0.775466,0.00415289,0.191019
neutral,0.000201495,0.00297931,0.000760255,0.000211083,0.0211697,0.972772,0.000222323,0.0016835
happy,0.000556266,0.0164881,0.000368954,0.00057013,0.969002,0.0100853,0.000967995,0.00196166
fear,0.384633,0.0042521,0.100513,0.41946,0.000773854,0.00235564,0.0465746,0.0414379
happy,9.71884e-06,6.19527e-05,1.64672e-05,5.43716e-06,0.999676,0.000194392,9.64148e-06,2.5969e-05
neutral,0.000286821,0.00545247,0.00136963,0.000255059,0.014101,0.975715,0.000183797,0.00263675
anger,0.751265,0.00333145,0.0869584,0.0743677,0.00102531,0.00193492,0.0639569,0.0171607
anger,0.36559,0.13635,0.331431,0.0458117,0.00330248,0.00221002,0.108022,0.00728327


In [69]:
best_model = aml.leader
# aml.score(test_h2o_frame, test_h2o_frame.MEDV)


In [70]:
h2o.save_model(model=best_model, path="./")

'/home/vorkov/Workspace/Emotion-Decetion-Service/learning/XGBoost_1_AutoML_2_20230909_43641'

In [24]:
h2o.load_model('/home/vorkov/Workspace/Emotion-Decetion-Service/learning/GLM_1_AutoML_1_20230909_00442')

Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_1_AutoML_1_20230909_00442


GLM Model: summary
    family       link         regularization               lambda_search                                                                   number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  -----------  -----------  ---------------------------  ------------------------------------------------------------------------------  ----------------------------  -----------------------------  ----------------------  ---------------------------------------------------------------------------------------
    multinomial  multinomial  Ridge ( lambda = 0.007732 )  nlambda = 30, lambda.max = 21.707, lambda.min = 0.007732, lambda.1se = 0.01062  8430                          8424                           205                     AutoML_1_20230909_00442_training_Key_Frame__upload_9b0ad4f7d1eb0254f20700a6c8f745c0.hex

ModelMetricsMultinomialGLM: glm
** Reported on train data. **

MSE: 0.1832485926285127
RMSE: 0.4280754520274582
LogLoss: 0.5848479289901717
Null degrees of freedom: 20190
Residual degrees of freedom: 11761
Null deviance: 63923.76920676409
Residual deviance: 23617.327619764204
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
anger    contempt    disgust    happy    neutral    surprise    Error      Rate
-------  ----------  ---------  -------  ---------  ----------  ---------  --------------
2146     420         0          18       46         529         0.320671   1,013 / 3,159
429      1948        0          47       84         349         0.318166   909 / 2,857
1        0           0          0        0          0           1          1 / 1
12       23          0          4407     553        43          0.125248   631 / 5,038
46       38          0          316      4614       103         0.0982998  503 / 5,117
383      360         0          104      325        2847        0.291615   1,172 / 4,019
3017     2789        0          4892     5622       3871        0.20945    4,229 / 20,191

Top-6 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.79055
2    0.942301
3    0.984349
4    0.997524
5    0.999851
6    1

ModelMetricsMultinomialGLM: glm
** Reported on cross-validation data. **

MSE: 0.18351216993959682
RMSE: 0.42838320454891415
LogLoss: 0.588139110499819
Null degrees of freedom: 20190
Residual degrees of freedom: 11761
Null deviance: Infinity
Residual deviance: 23750.499560828222
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
anger    contempt    disgust    happy    neutral    surprise    Error      Rate
-------  ----------  ---------  -------  ---------  ----------  ---------  --------------
2125     436         0          17       46         535         0.327319   1,034 / 3,159
407      1972        0          46       84         348         0.309765   885 / 2,857
1        0           0          0        0          0           1          1 / 1
11       22          0          4405     554        46          0.125645   633 / 5,038
46       38          0          316      4610       107         0.0990815  507 / 5,117
389      361         0          100      323        2846        0.291864   1,173 / 4,019
2979     2829 

In [71]:
import numpy as np
from sklearn. metrics import f1_score

pred = aml.predict(test_h2o_frame)
pd=h2o.as_list(pred) 
f1_score(test['col1404'], pd["predict"].tolist(), average="weighted")

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


0.550724958244778

In [72]:
test['col1404']

10830    surprise
28509        fear
13902     neutral
14910     neutral
15942     neutral
           ...   
1854     contempt
23327         sad
19950       anger
17097       anger
12642     neutral
Name: col1404, Length: 4327, dtype: object

In [73]:
pd["predict"].tolist()

['surprise',
 'fear',
 'neutral',
 'neutral',
 'happy',
 'fear',
 'happy',
 'neutral',
 'anger',
 'anger',
 'neutral',
 'contempt',
 'surprise',
 'surprise',
 'happy',
 'sad',
 'fear',
 'disgust',
 'fear',
 'neutral',
 'anger',
 'neutral',
 'fear',
 'neutral',
 'happy',
 'neutral',
 'surprise',
 'surprise',
 'neutral',
 'sad',
 'surprise',
 'disgust',
 'neutral',
 'anger',
 'happy',
 'sad',
 'happy',
 'fear',
 'fear',
 'disgust',
 'happy',
 'happy',
 'happy',
 'disgust',
 'disgust',
 'fear',
 'disgust',
 'anger',
 'neutral',
 'contempt',
 'surprise',
 'surprise',
 'neutral',
 'fear',
 'contempt',
 'happy',
 'anger',
 'surprise',
 'contempt',
 'surprise',
 'disgust',
 'sad',
 'disgust',
 'neutral',
 'fear',
 'happy',
 'fear',
 'neutral',
 'anger',
 'contempt',
 'anger',
 'sad',
 'happy',
 'surprise',
 'disgust',
 'neutral',
 'anger',
 'happy',
 'neutral',
 'neutral',
 'neutral',
 'fear',
 'contempt',
 'anger',
 'surprise',
 'surprise',
 'contempt',
 'neutral',
 'neutral',
 'surprise',
 

In [ ]:
f1_score(test['col1404'], pd["predict"].tolist(), average="weighted")